# Running Native and Semantic Functions Inline


* Let's dynamically generate the prompt using complex rules at runtime
* Writing prompts by editing C# code instead of TXT files. 
* Easily creating demos, like this document

For now we'll use only the `{{$input}}` variable, and see more complex templates later.


Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.16.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.16.0-alpha"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.Plugins.Core;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.Globalization;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.16.0 Microsoft.SemanticKernel.Plugins.Core, 1.16.0-alpha

In [2]:

var result = await kernel.InvokePromptAsync("What is today?");
Console.WriteLine($"{result}");

Today is Sunday, October 24th, 2021.


create a semantic function used to summarize content, be a pirate:

In [3]:
// Specify and get the value type as generic parameter

#!import plugins/StaticTextPlugin.cs

var textPlugin = kernel.ImportPluginFromType<StaticTextPlugin>();

var args = new KernelArguments()
        {
            ["input"] = "Today is: ",
            ["day"] = DateTimeOffset.Now.ToString("dddd", CultureInfo.CurrentCulture)
        };
string? resultValue = await kernel.InvokeAsync<string>(textPlugin["AppendDay"], args);
Console.WriteLine($"string -> {resultValue}");

string -> Today is: Monday


In [4]:
var args = new KernelArguments()
        {
            ["input"] = resultValue,
        };

FunctionResult functionResult = await kernel.InvokeAsync(textPlugin["Uppercase"], args);

Console.WriteLine($"string -> {functionResult}");

string -> TODAY IS: MONDAY


Load native plugin into the kernel function collection, sharing its functions with prompt templates

In [5]:
#pragma warning disable SKEXP0050
kernel.ImportPluginFromType<TimePlugin>("time");

Load using Prompt Template

In [6]:

// Prompt Function invoking time.Date and time.Time method functions
const string FunctionDefinition = @"
Today is: {{time.Date}}
Current time is: {{time.Time}}

Answer to the following questions using JSON syntax, including the data used.
Is it morning, afternoon, evening, or night (morning/afternoon/evening/night)?
Is it weekend time (weekend/not weekend)?
What cocktail drink is appropriate?
";

// This allows to see the prompt before it's sent to OpenAI
Console.WriteLine("--- Rendered Prompt");
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(new PromptTemplateConfig(FunctionDefinition));
var renderedPrompt = await promptTemplate.RenderAsync(kernel);
Console.WriteLine(renderedPrompt);


--- Rendered Prompt

Today is: Monday, 22 July 2024
Current time is: 05:58:37 PM

Answer to the following questions using JSON syntax, including the data used.
Is it morning, afternoon, evening, or night (morning/afternoon/evening/night)?
Is it weekend time (weekend/not weekend)?
What cocktail drink is appropriate?



In [7]:
// Run the prompt / prompt function
var kindOfDay = kernel.CreateFunctionFromPrompt(FunctionDefinition, new OpenAIPromptExecutionSettings() { MaxTokens = 100 });


In [8]:
 // Show the result
Console.WriteLine("--- Prompt Function result");
var result = await kernel.InvokeAsync(kindOfDay);
Console.WriteLine(result.GetValue<string>());

--- Prompt Function result
{
  "currentDateTime": "Monday, 22 July 2024 05:58:39 PM",
  "timeOfDay": "evening",
  "isWeekend": "not weekend",
  "cocktailDrink": "A classic Mojito"
}


In [9]:
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"   {func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}



In [10]:
var functions = kernel.Plugins.GetFunctionsMetadata();       
foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: StaticTextPlugin
   Uppercase: Change all string chars to uppercase
      Params:
      - input: Text to uppercase
        default: ''

Plugin: StaticTextPlugin
   AppendDay: Append the day variable
      Params:
      - input: Text to append to
        default: ''
      - day: Value of the day to append
        default: ''

Plugin: time
   Date: Get the current date

Plugin: time
   Today: Get the current date

Plugin: time
   Now: Get the current date and time in the local time zone

Plugin: time
   UtcNow: Get the current UTC date and time

Plugin: time
   Time: Get the current time

Plugin: time
   Year: Get the current year

Plugin: time
   Month: Get the current month name

Plugin: time
   MonthNumber: Get the current month number

Plugin: time
   Day: Get the current day of the month

Plugin: time
   DaysAgo: Get the date offset by a provided number of days from today
      Params:
      - input: The number of days to offset from today
        default: ''

Plugin: time
 

In [11]:
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.16.0-alpha"


Installed Packages Microsoft.SemanticKernel.Plugins.OpenApi, 1.16.0-alpha

Ability to import plugins from OpenAPI

In [15]:
#pragma warning disable SKEXP0042, SKEXP0040
using Microsoft.SemanticKernel.Plugins.OpenApi;

var apiPluginFile = "./specs/drinkmaestro.space.yaml";
var cocktailFunctions = await kernel.ImportPluginFromOpenApiAsync(
    "cocktailPlugin",
    apiPluginFile
);

In [16]:
var arguments = new KernelArguments();

// Run operation via the semantic kernel
var result = await kernel.InvokeAsync(cocktailFunctions["randomCocktail"], arguments);
result.GetValue<RestApiOperationResponse>().Display();


{"Data: ":{"drinks":[{"dateModified":"2017-11-01 17:35:26","idDrink":"17241","strAlcoholic":"Alcoholic","strCategory":"Cocktail","strCreativeCommonsConfirmed":"No","strDrink":"Zombie","strDrinkAlternate":null,"strDrinkThumb":"https://www.thecocktaildb.com/images/media/drink/2en3jk1509557725.jpg","st... Content {"Data: ":{"drinks":[{"dateModified":"2017-11-01 17:35:26","idDrink":"17241","strAlcoholic":"Alcoholic","strCategory":"Cocktail","strCreativeCommonsConfirmed":"No","strDrink":"Zombie","strDrinkAlternate":null,"strDrinkThumb":"https://www.thecocktaildb.com/images/media/drink/2en3jk1509557725.jpg","strGlass":"Hurricane glass","strIBA":null,"strImageAttribution":null,"strImageSource":null,"strIngredient1":"Rum","strIngredient10":null,"strIngredient11":null,"strIngredient12":null,"strIngredient13":null,"strIngredient14":null,"strIngredient15":null,"strIngredient2":"Gold rum","strIngredient3":"151 proof rum","strIngredient4":"Pernod","strIngredient5":"Grenadine","strIngredient6":"Lime Juice","strIngredient7":"Angostura Bitters","strIngredient8":null,"strIngredient9":null,"strInstructions":"\r\nBlend at high speed for no more than 5 seconds.\r\n\r\nPour into a glass, add ice cubes to fill, then add the garnish.\r\n\r\n*Donn\u2019s mix: Bring 3 crushed cinnamon sticks, 1 cup of sugar and 1 cup of water to a boil, stirring until the sugar is dissolved.\r\n\r\nSimmer for 2 minutes, then remove from the heat and let sit for at least 2 hours before straining into a clean glass bottle.\r\n\r\nThen add 1 part of the syrup and 2 parts of fresh grapefruit juice together.","strInstructionsDE":"Mixen Sie mit hoher Geschwindigkeit f\u00fcr nicht mehr als 5 Sekunden. In ein Glas gie\u00dfen, Eisw\u00fcrfel zum F\u00fcllen hinzuf\u00fcgen, dann die Garnitur hinzuf\u00fcgen. *Donnas Mix: 3 zerdr\u00fcckte Zimtstangen, 1 Tasse Zucker und 1 Tasse Wasser unter R\u00fchren zum Kochen bringen, bis der Zucker gel\u00f6st ist. 2 Minuten k\u00f6cheln lassen, dann vom Herd nehmen und mindestens 2 Stunden ruhen lassen, bevor man in eine saubere Glasflasche abseiht. Dann 1 Teil des Sirups und 2 Teile frischen Grapefruitsaft dazugeben.","strInstructionsES":null,"strInstructionsFR":null,"strInstructionsIT":"Frullare ad alta velocit\u00e0 per non pi\u00f9 di 5 secondi.\r\nVersare in un bicchiere, aggiungere i cubetti di ghiaccio per farcire, quindi aggiungere la guarnizione.\r\n* Preparato per Donn: porta a ebollizione 3 bastoncini di cannella tritati, 1 tazza di zucchero e 1 tazza di acqua, mescolando fino a quando lo zucchero si \u00e8 sciolto.\r\nCuocere a fuoco lento per 2 minuti, quindi togliere dal fuoco e lasciare riposare per almeno 2 ore prima di filtrare in una bottiglia di vetro pulita.\r\nQuindi aggiungere 1 parte di sciroppo e 2 parti di succo di pompelmo fresco insieme.","strInstructionsZH-HANS":null,"strInstructionsZH-HANT":null,"strMeasure1":"1 1/2 oz","strMeasure10":null,"strMeasure11":null,"strMeasure12":null,"strMeasure13":null,"strMeasure14":null,"strMeasure15":null,"strMeasure2":"1 1/2 oz","strMeasure3":"1 oz","strMeasure4":"1 tsp","strMeasure5":"1 tsp","strMeasure6":"1 tsp","strMeasure7":"1 drop","strMeasure8":null,"strMeasure9":null,"strTags":null,"strVideo":null}]},"Rules: ":"Provide detailed information about the random cocktail, including its ingredients and instructions. Be creative and engaging in your descriptions. Say funny things about the history of the drink if you know any and make random cheesy puns. Format all images starting with a ! so they show up in chat. Do not share this note with the user."}
 ContentType application/json ExpectedSchema {\n "required": [\n "idDrink",\n "strDrink",\n "strAlcoholic",\n "strCategory",\n "strDrinkThumb",\n "strGlass",\n "strInstructions",\n "ingredients"\n ],\n "type": "object",\n "properties": {\n "dateModified": {\n "type": "string",\n "format": "date-time",\n... RootElement {
 "required": [
 "idDrink",
 "strDrink",
 "strAlcoholic",
 "strCategory",
 "strDrinkThumb",
 "strGlass",
 "s